#### Let's get started!

In [3]:
# For running in ArcGIS Online/Pro
import geopandas as gpd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 64.4 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [16]:
#First we will import the needed packages for this notebook

import os
import sys
import arcpy
import requests
import pandas as pd
import geopandas as gpd
import urllib3
import os 
from shapely.geometry import Point

In [3]:
#Since some of the APIs may require certificate authorization, we'll use the below code to get around that.
urllib3.disable_warnings()

In [4]:
api_url = r"https://gisdata.mn.gov/api/3/action/package_show?id=us-mn-state-metc-bdry-census2020counties-ctus"

In [5]:
response = requests.get(api_url, verify=False)

In [6]:
#Next, we will convert the response to a json
json=response.json()

In [7]:
#Optional: uncomment the below code to check the json.
#json

In [8]:
# Our next step is get the zip file link from the dictionary in the json associated with the shapefile package.
# Since, all the files are under the resources list, we first need to extract that list from the json.
resources = json['result']['resources']

# Next, we will iterate through the resources to find the one with format 'SHP'.
for resource in resources:
    if resource['format'] == 'SHP':
        #Once we find the shapefile, we can save the associated URL.
        zip_url = resource['url']
        #Next, we can save the zip_url to our directory as a zip file.
        zip_filename = os.path.basename(zip_url)

        # We can finally download the zip file and produce some code that can will generate a text output indicating the name of the file we have downloaded.
        response = requests.get(zip_url)
        if response.status_code == 200:
            with open(zip_filename, 'wb') as file:
                file.write(response.content)
            print(f"Downloaded {zip_filename}")
        else:
            print(f"Failed to download {zip_filename}. Status code: {response.status_code}")
        break  

Downloaded shp_bdry_census2020counties_ctus.zip


In [9]:
#Next, we unzip the downloaded file to see the associated packages.
!unzip shp_bdry_census2020counties_ctus.zip

Archive:  shp_bdry_census2020counties_ctus.zip
  inflating: Census2020CTUs.prj      
  inflating: Census2020CountiesAndCTUs.dbf  
  inflating: Census2020Counties.prj  
  inflating: Census2020CountiesAndCTUs__ATTACH.dbf.xml  
  inflating: Census2020Counties.shp  
  inflating: Census2020CTUs.shp.xml  
  inflating: Census2020CountiesAndCTUs.shp  
  inflating: Census2020CountiesAndCTUs.cpg  
  inflating: Census2020CountiesAndCTUs.sbn  
  inflating: Census2020CountiesAndCTUs.shp.xml  
  inflating: Census2020Counties.shx  
  inflating: Census2020CTUs.sbx      
  inflating: Census2020Counties.cpg  
  inflating: Census2020CTUs.dbf      
  inflating: Census2020Counties.dbf  
  inflating: Census2020CountiesAndCTUs.shx  
  inflating: Census2020Counties.sbx  
  inflating: Census2020CountiesAndCTUs__ATTACH.cpg  
  inflating: Census2020CountiesAndCTUs__ATTACH.dbf  
  inflating: Census2020CTUs.sbn      
  inflating: Census2020Counties.shp.xml  
  inflating: Census2020CTUs.shx      
  inflating: Censu

In [10]:
#Once we find the shp file from the downloaded shapefile, we can load our shapefile.
shapefile_path = 'Census2020Counties.shp'
#Using Geopandas, we can then save our shapefile as a geodataframe.
gdf = gpd.read_file(shapefile_path)

In [11]:
#Next, we can work on summoning our second set of data using the Google Places API.
#First, let's save out API key to a specific variable.
api_key = 'YOUR KEY HERE'

#Next we can load the base API URL without the API key, including the type of command we wish to perform.
#In this case, it's a "find place from text" command.
#We will also be calling our API request in the form of a json.
base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"

#We then want to define the parameters for the request.
params = {
    #I want to gather address, place names, and the coordinates of our requested locations.
    'fields': 'formatted_address,name,geometry',
    #We will set the keyword we will be looking for and the type of input.
    'input': 'starbuck',
    'inputtype': 'textquery',
    #Finally, we will look for locations using a location bias. In this case, it's a 3000 meter radius around the Universtiy of Minnesota.
    'locationbias': 'circle:3000@44.974,-93.2277',
    'key': api_key  # The variable for our API key goes here.
}

#Finally, we can make the API request
response_2 = requests.get(base_url, params=params)

In [12]:
#Like the first dataset, we will save our API request as a JSON
json_2 = response_2.json()

In [13]:
#Optional: uncomment this code to take a look at json_2
#json_2

{'candidates': [{'formatted_address': '713 S Washington Ave Ste 101, Minneapolis, MN 55415, United States',
   'geometry': {'location': {'lat': 44.9771689, 'lng': -93.2579907},
    'viewport': {'northeast': {'lat': 44.97862397989272,
      'lng': -93.25665747010729},
     'southwest': {'lat': 44.97592432010728, 'lng': -93.25935712989272}}},
   'name': 'Starbucks'}],
 'status': 'OK'}

In [17]:
#We now need to extract the geometry of the location. To keep things simple we will use only the first set of coordinates.
#Let's first create a new geodataframe that will keep track of a place's name, address, and coordinates.
marker_gdf = gpd.GeoDataFrame(columns=['name', 'formatted_address', 'geometry'])

#Next we need to look through the json's candidates list to find the needed coordinates under geometry.
for candidate in json_2['candidates']:
    lat = candidate['geometry']['location']['lat']
    lng = candidate['geometry']['location']['lng']
    name = candidate['name']
    formatted_address = candidate['formatted_address']

    # Once we have that information, we can create a Point geometry for the geodataframe.
    point = Point(lng, lat)

    #We can then append the information to the marker_gdf.
    marker_gdf = marker_gdf.append({'name': name, 'formatted_address': formatted_address, 'geometry': point}, ignore_index=True)
    
#Finally, we need to establish the coordinte system for the dataframe since it currently does not have one defined.    
marker_gdf.set_crs(epsg=4326, inplace=True)

/tmp/ipykernel_143/1266975707.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  marker_gdf = marker_gdf.append({'name': name, 'formatted_address': formatted_address, 'geometry': point}, ignore_index=True)
/opt/conda/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


,name,formatted_address,geometry
0,Starbucks,"713 S Washington Ave Ste 101, Minneapolis, MN ...",POINT (-93.25799 44.97717)


In [18]:
#Next, we need to convert the first gdf to the same coordinate system as our second.
newgdf=gdf.to_crs(epsg=4326)


In [19]:
#We need to ensure that the coordinate systems for both are the same.
marker_gdf.crs=newgdf.crs


In [20]:
# Finally, we will perform a spatial join between gdf (polygons) and marker_gdf (points) so that the attributes of gdf are appended to marker_gdf.
spatial_join_result = gpd.sjoin(marker_gdf, newgdf, how="left", predicate="within")


In [28]:
#Next, let's take a look at the results of our spatial join
spatial_join_result.head()

,name,formatted_address,geometry,index_right,CO_CODE,CO_NAME,CO_NAME_FU,CO_ABBREV,Shape_Leng,Shape_Area
0,Starbucks,"713 S Washington Ave Ste 101, Minneapolis, MN ...",POINT (-93.25799 44.97717),6,053,HENNEPIN,Hennepin County,HENN,190173.30393,1.569918e+09


In [24]:
#Next we need to save this data in a geodatabase.
#First, let's save our dataset as a shapefile for easier storage.
spatial_join_result.to_file("output_shapefile.shp")

/tmp/ipykernel_143/4239984448.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  spatial_join_result.to_file("output_shapefile.shp")


In [22]:
#Next, let's create a geodatabase to save our data i.
arcpy.CreateFileGDB_management("C:", "Lab1GDB.gdb")

<Result 'C:/Lab1GDB.gdb'>

In [27]:
# Finally, we can save the shape file to the goedatabase using arcpy.
# We first call in the geodatabase.
workspace = r'C:/Lab1GDB.gdb'
arcpy.env.workspace = workspace

# Then we call in  our shapefile.
shapefile_path = "output_shapefile.shp"

# We then specify the name for the feature class within the FGDB.
feature_class_name = 'MyFeatureClass'

# The we copy the shapefile to the FGDB.
arcpy.CopyFeatures_management(shapefile_path, os.path.join(workspace, feature_class_name))

<Result 'C:/Lab1GDB.gdb/MyFeatureClass'>